In [ ]:
# import all the needed libraries
import pandas as pd
from depiction.models.paccmann import PaccMannSmiles, PaccMannCellLine

In [ ]:
# initialize some data
# drugs
drugs = pd.read_csv('../data/paccmann/gdsc.smi', sep='\t', index_col=1)
# a drug
drug = drugs.iloc[3].item()
# some drugs
some_drugs = list(drugs.values[:, 0][:100])
# cell lines
cell_lines = pd.read_csv('../data/paccmann/gdsc.csv.gz', index_col=0)
# a cell line
cell_line = cell_lines.iloc[5][3:].values
# some cell lines
some_cell_lines = list(cell_lines.values[:200, 3:])

In [ ]:
# instantiate paccmann for prediction on SMILES given a cell line
paccmann_smiles = PaccMannSmiles(cell_line)

In [ ]:
# predict on SMILES
results_smiles = paccmann_smiles.predict(some_drugs)
results_smiles

In [ ]:
# instantiate paccmann for prediction on cell lines given a SMILES
paccmann_cell_line = PaccMannCellLine(drug)

In [ ]:
# predict on cell lines
results_cell_line = paccmann_cell_line.predict(some_cell_lines)
results_cell_line